# Enhanced Stock Forecasting with Multiple LSTM Models and Advanced Sentiment Analysis

This comprehensive notebook implements:
1. **Multiple Sentiment Analysis Models** (FinBERT, VADER, TextBlob, Ensemble)
2. **4-5 Different LSTM Architectures** with optimized configurations
3. **Dynamic Epoch Testing** (25, 50, 75, 100 epochs)
4. **Advanced Feature Engineering** with enhanced technical indicators
5. **Comprehensive Visualizations** and performance comparisons
6. **Multiple Detailed Reports** for thorough analysis


In [1]:
# Enhanced imports for multiple sentiment models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import warnings
from datetime import datetime, timedelta
import re

# Machine Learning Libraries
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Multiple NLP Libraries for sentiment analysis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Configuration
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print("Enhanced LSTM Analysis with Multiple Models Initialized!")


/opt/anaconda3/envs/tf_metal/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


TensorFlow version: 2.19.0
Enhanced LSTM Analysis with Multiple Models Initialized!


In [2]:
# Install additional sentiment analysis libraries if needed
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    print("VADER Sentiment available")
except ImportError:
    print("Installing VADER Sentiment...")
    import subprocess
    subprocess.check_call(["pip", "install", "vaderSentiment"])
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

try:
    from textblob import TextBlob
    print("TextBlob available")
except ImportError:
    print("Installing TextBlob...")
    import subprocess
    subprocess.check_call(["pip", "install", "textblob"])
    from textblob import TextBlob


Installing VADER Sentiment...
  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata (572 bytes)
Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)
Installing TextBlob...
  Using cached textblob-0.19.0-py3-none-any.whl.metadata (4.4 kB)
Using cached textblob-0.19.0-py3-none-any.whl (624 kB)


## 1. Enhanced Data Loading and Stock Selection


In [3]:
def load_and_analyze_news_data(file_path='news_data.csv'):
    """
    Enhanced news data loading with better analysis
    """
    print("Loading news data...")
    df = pd.read_csv(file_path)
    
    try:
        df['Date'] = pd.to_datetime(df['date'], format='mixed', utc=True)
    except Exception as e:
        print(f"Error converting dates: {e}")
        df['Date'] = pd.to_datetime(df['date'], format='mixed', utc=True, errors='coerce')
        df = df.dropna(subset=['Date'])
    
    print(f"Dataset shape: {df.shape}")
    
    # Enhanced stock analysis with more metrics
    stock_analysis = df.groupby('stock').agg({
        'date': ['count', 'min', 'max'],
        'title': lambda x: len(' '.join(x).split())  # Total word count
    }).round(2)
    
    stock_analysis.columns = ['Article_Count', 'Start_Date', 'End_Date', 'Total_Words']
    stock_analysis = stock_analysis.sort_values('Article_Count', ascending=False)
    
    # Calculate date range in days
    stock_analysis['Date_Range_Days'] = (
        pd.to_datetime(stock_analysis['End_Date'], utc=True) -
        pd.to_datetime(stock_analysis['Start_Date'], utc=True)
    ).dt.days
    
    # Calculate articles per day
    stock_analysis['Articles_Per_Day'] = (
        stock_analysis['Article_Count'] / stock_analysis['Date_Range_Days']
    ).round(2)
    
    top_10_stocks = stock_analysis.head(10)
    print("\nTop 10 stocks with enhanced metrics:")
    print(top_10_stocks)
    
    # Select 4th most frequent stock (as in original)
    target_stock = top_10_stocks.index[3]
    print(f"\n🎯 Selected target stock: {target_stock}")
    print(f"   Articles: {top_10_stocks.loc[target_stock, 'Article_Count']}")
    print(f"   Articles per day: {top_10_stocks.loc[target_stock, 'Articles_Per_Day']}")
    
    return df, target_stock

# Load data
news_df, TARGET_STOCK = load_and_analyze_news_data()


Loading news data...
Error converting dates: Unknown datetime string format, unable to parse: AAN, at position 2299
Dataset shape: (1397891, 5)

Top 10 stocks with enhanced metrics:
       Article_Count                 Start_Date                   End_Date  \
stock                                                                        
MRK             3334  2009-07-27 08:33:00-04:00  2020-06-11 10:22:00-04:00   
MS              3242  2010-01-20 06:56:00-05:00  2020-06-11 11:03:00-04:00   
MU              3144  2011-04-20 19:20:00-04:00  2020-06-10 07:35:00-04:00   
NVDA            3133  2011-03-03 10:06:00-05:00  2020-06-10 12:37:00-04:00   
QQQ             3100  2011-03-16 22:07:00-04:00  2020-06-10 12:12:00-04:00   
M               3078  2009-06-16 08:14:00-04:00  2020-06-11 10:34:00-04:00   
EBAY            3021  2011-10-13 06:01:00-04:00  2020-06-10 16:20:00-04:00   
NFLX            3009  2016-08-23 08:41:00-04:00  2020-06-10 16:20:00-04:00   
GILD            2969  2009-08-17 06:40

## 2. Multiple Sentiment Analysis Models


In [4]:
class MultiSentimentAnalyzer:
    """
    Enhanced sentiment analyzer using multiple models
    """
    
    def __init__(self):
        print("Initializing multiple sentiment models...")
        
        # FinBERT model
        self.finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
        self.finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
        
        # VADER analyzer
        self.vader_analyzer = SentimentIntensityAnalyzer()
        
        print("All sentiment models loaded successfully!")
    
    def clean_text(self, text):
        """Enhanced text cleaning"""
        if text is None or pd.isna(text):
            return ""
        text = str(text)
        text = re.sub(r'[^\w\s.]', '', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def analyze_finbert(self, text):
        """FinBERT sentiment analysis"""
        try:
            cleaned_text = self.clean_text(text)
            if not cleaned_text:
                return {'label': 'neutral', 'score': 0.0}
            
            inputs = self.finbert_tokenizer(cleaned_text, return_tensors="pt", 
                                          padding=True, truncation=True, max_length=512)
            
            with torch.no_grad():
                outputs = self.finbert_model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            
            predicted_class = torch.argmax(predictions, dim=1).item()
            confidence = torch.max(predictions, dim=1)[0].item()
            
            sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
            label = sentiment_map[predicted_class]
            
            # Convert to numerical score
            score_map = {'negative': -1, 'neutral': 0, 'positive': 1}
            score = score_map[label] * confidence
            
            return {'label': label, 'score': score}
        
        except Exception as e:
            return {'label': 'neutral', 'score': 0.0}
    
    def analyze_vader(self, text):
        """VADER sentiment analysis"""
        try:
            cleaned_text = self.clean_text(text)
            if not cleaned_text:
                return {'label': 'neutral', 'score': 0.0}
            
            scores = self.vader_analyzer.polarity_scores(cleaned_text)
            compound_score = scores['compound']
            
            # Classify based on compound score
            if compound_score >= 0.05:
                label = 'positive'
            elif compound_score <= -0.05:
                label = 'negative'
            else:
                label = 'neutral'
            
            return {'label': label, 'score': compound_score}
        
        except Exception as e:
            return {'label': 'neutral', 'score': 0.0}
    
    def analyze_textblob(self, text):
        """TextBlob sentiment analysis"""
        try:
            cleaned_text = self.clean_text(text)
            if not cleaned_text:
                return {'label': 'neutral', 'score': 0.0}
            
            blob = TextBlob(cleaned_text)
            polarity = blob.sentiment.polarity
            
            # Classify based on polarity
            if polarity > 0.1:
                label = 'positive'
            elif polarity < -0.1:
                label = 'negative'
            else:
                label = 'neutral'
            
            return {'label': label, 'score': polarity}
        
        except Exception as e:
            return {'label': 'neutral', 'score': 0.0}
    
    def analyze_ensemble(self, text, weights=None):
        """
        Ensemble sentiment analysis combining all models
        """
        if weights is None:
            weights = {'finbert': 0.5, 'vader': 0.3, 'textblob': 0.2}  # FinBERT gets highest weight
        
        # Get individual predictions
        finbert_result = self.analyze_finbert(text)
        vader_result = self.analyze_vader(text)
        textblob_result = self.analyze_textblob(text)
        
        # Calculate weighted ensemble score
        ensemble_score = (
            weights['finbert'] * finbert_result['score'] +
            weights['vader'] * vader_result['score'] +
            weights['textblob'] * textblob_result['score']
        )
        
        # Classify ensemble result
        if ensemble_score > 0.1:
            ensemble_label = 'positive'
        elif ensemble_score < -0.1:
            ensemble_label = 'negative'
        else:
            ensemble_label = 'neutral'
        
        return {
            'ensemble_label': ensemble_label,
            'ensemble_score': ensemble_score,
            'finbert': finbert_result,
            'vader': vader_result,
            'textblob': textblob_result
        }

# Initialize the multi-sentiment analyzer
sentiment_analyzer = MultiSentimentAnalyzer()


Initializing multiple sentiment models...
All sentiment models loaded successfully!


In [5]:
def process_enhanced_sentiment(news_df, target_stock, analyzer):
    """
    Process news sentiment using multiple models
    """
    print(f"\n📰 Processing enhanced sentiment for {target_stock}...")
    
    # Filter news for target company
    company_news = news_df[news_df['stock'] == target_stock].copy()
    print(f"Found {len(company_news)} news articles for {target_stock}")
    
    if company_news.empty:
        return None
    
    # Process dates
    company_news['date'] = pd.to_datetime(company_news['date'], utc=True, errors='coerce')
    company_news = company_news.dropna(subset=['date'])
    company_news['Date'] = company_news['date'].dt.date
    
    print("Analyzing sentiment with multiple models...")
    
    # Apply ensemble sentiment analysis
    sentiment_results = []
    for idx, row in company_news.iterrows():
        result = analyzer.analyze_ensemble(row['title'])
        sentiment_results.append(result)
    
    # Extract results into separate columns
    company_news['Ensemble_Label'] = [r['ensemble_label'] for r in sentiment_results]
    company_news['Ensemble_Score'] = [r['ensemble_score'] for r in sentiment_results]
    company_news['FinBERT_Label'] = [r['finbert']['label'] for r in sentiment_results]
    company_news['FinBERT_Score'] = [r['finbert']['score'] for r in sentiment_results]
    company_news['VADER_Label'] = [r['vader']['label'] for r in sentiment_results]
    company_news['VADER_Score'] = [r['vader']['score'] for r in sentiment_results]
    company_news['TextBlob_Label'] = [r['textblob']['label'] for r in sentiment_results]
    company_news['TextBlob_Score'] = [r['textblob']['score'] for r in sentiment_results]
    
    # Display sentiment distribution for each model
    print(f"\n📊 Sentiment Distribution Comparison:")
    models = ['Ensemble', 'FinBERT', 'VADER', 'TextBlob']
    for model in models:
        label_col = f'{model}_Label'
        counts = company_news[label_col].value_counts()
        print(f"\n{model}:")
        for sentiment, count in counts.items():
            percentage = (count / len(company_news)) * 100
            print(f"  {sentiment}: {count} ({percentage:.1f}%)")
    
    return company_news

# Process sentiment
company_sentiment_df = process_enhanced_sentiment(news_df, TARGET_STOCK, sentiment_analyzer)



📰 Processing enhanced sentiment for NVDA...
Found 3133 news articles for NVDA
Analyzing sentiment with multiple models...


KeyboardInterrupt: 

In [6]:
def aggregate_enhanced_sentiment(sentiment_df):
    """
    Aggregate enhanced sentiment scores on a daily basis
    """
    if sentiment_df is None or sentiment_df.empty:
        return None
    
    print("\n📊 Aggregating enhanced daily sentiment scores...")
    
    # Group by date and calculate comprehensive metrics
    daily_sentiment = sentiment_df.groupby('Date').agg({
        'Ensemble_Score': ['mean', 'std', 'min', 'max'],
        'FinBERT_Score': ['mean', 'std'],
        'VADER_Score': ['mean', 'std'],
        'TextBlob_Score': ['mean', 'std'],
        'Ensemble_Label': ['count', lambda x: (x == 'positive').sum(), lambda x: (x == 'negative').sum()]
    }).round(4)
    
    # Flatten column names
    daily_sentiment.columns = [
        'Ensemble_Mean', 'Ensemble_Std', 'Ensemble_Min', 'Ensemble_Max',
        'FinBERT_Mean', 'FinBERT_Std',
        'VADER_Mean', 'VADER_Std', 
        'TextBlob_Mean', 'TextBlob_Std',
        'News_Count', 'Positive_Count', 'Negative_Count'
    ]
    
    # Calculate additional metrics
    daily_sentiment['Neutral_Count'] = daily_sentiment['News_Count'] - daily_sentiment['Positive_Count'] - daily_sentiment['Negative_Count']
    daily_sentiment['Sentiment_Ratio'] = (
        (daily_sentiment['Positive_Count'] - daily_sentiment['Negative_Count']) / 
        daily_sentiment['News_Count']
    ).fillna(0)
    
    # Calculate sentiment volatility (standard deviation of ensemble scores)
    daily_sentiment['Sentiment_Volatility'] = daily_sentiment['Ensemble_Std'].fillna(0)
    
    print(f"Enhanced daily sentiment data shape: {daily_sentiment.shape}")
    print(f"Date range: {daily_sentiment.index.min()} to {daily_sentiment.index.max()}")
    
    return daily_sentiment

# Aggregate enhanced sentiment
daily_sentiment_df = aggregate_enhanced_sentiment(company_sentiment_df)

if daily_sentiment_df is not None:
    print("\nSample enhanced daily sentiment data:")
    display_cols = ['Ensemble_Mean', 'FinBERT_Mean', 'VADER_Mean', 'TextBlob_Mean', 'News_Count', 'Sentiment_Ratio']
    print(daily_sentiment_df[display_cols].head(10))



📊 Aggregating enhanced daily sentiment scores...
Enhanced daily sentiment data shape: (1195, 16)
Date range: 2011-03-03 to 2020-06-10

Sample enhanced daily sentiment data:
            Ensemble_Mean  FinBERT_Mean  VADER_Mean  TextBlob_Mean  \
Date                                                                 
2011-03-03        -0.2316        0.0000     -0.5719        -0.3000   
2011-03-07         0.3387        0.9015     -0.2903        -0.1250   
2011-03-08        -0.0364       -0.0620     -0.0180         0.0000   
2011-03-09         0.4571        0.9143      0.0000         0.0000   
2011-03-10         0.5466        0.7718      0.2023         0.5000   
2011-03-15         0.2664        0.6175     -0.0496        -0.1375   
2011-03-16         0.2858        0.6177     -0.0987         0.0327   
2011-03-23         0.5641        0.8068      0.2023         0.5000   
2011-03-24         0.4627        0.9255      0.0000         0.0000   
2011-03-25         0.5604        0.7994      0.2023     

## 3. Multiple LSTM Model Architectures


In [7]:
class MultiLSTMModels:
    """
    Collection of different LSTM architectures for comparison
    """
    
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.models = {}
        
    def build_standard_lstm(self, name="Standard_LSTM"):
        """Standard stacked LSTM model"""
        model = Sequential([
            LSTM(units=100, return_sequences=True, input_shape=self.input_shape),
            Dropout(0.2),
            LSTM(units=100, return_sequences=True),
            Dropout(0.2),
            LSTM(units=50),
            Dropout(0.2),
            Dense(units=1, activation='linear')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mean_squared_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError()]
        )
        
        self.models[name] = model
        return model
    
    def build_bidirectional_lstm(self, name="Bidirectional_LSTM"):
        """Bidirectional LSTM model"""
        model = Sequential([
            Bidirectional(LSTM(units=50, return_sequences=True), input_shape=self.input_shape),
            Dropout(0.3),
            Bidirectional(LSTM(units=50, return_sequences=True)),
            Dropout(0.3),
            LSTM(units=50),
            Dropout(0.2),
            Dense(units=1, activation='linear')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mean_squared_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError()]
        )
        
        self.models[name] = model
        return model
    
    def build_deep_lstm(self, name="Deep_LSTM"):
        """Deep LSTM with more layers"""
        model = Sequential([
            LSTM(units=128, return_sequences=True, input_shape=self.input_shape),
            Dropout(0.2),
            LSTM(units=128, return_sequences=True),
            Dropout(0.2),
            LSTM(units=64, return_sequences=True),
            Dropout(0.2),
            LSTM(units=64, return_sequences=True),
            Dropout(0.2),
            LSTM(units=32),
            Dropout(0.2),
            Dense(units=1, activation='linear')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.0005),
            loss='mean_squared_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError()]
        )
        
        self.models[name] = model
        return model
    
    def build_wide_lstm(self, name="Wide_LSTM"):
        """Wide LSTM with more units per layer"""
        model = Sequential([
            LSTM(units=200, return_sequences=True, input_shape=self.input_shape),
            Dropout(0.3),
            LSTM(units=150, return_sequences=True),
            Dropout(0.3),
            LSTM(units=100),
            Dropout(0.2),
            Dense(units=50, activation='relu'),
            Dropout(0.2),
            Dense(units=1, activation='linear')
        ])
        
        model.compile(
            optimizer=RMSprop(learning_rate=0.001),
            loss='mean_squared_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError()]
        )
        
        self.models[name] = model
        return model
    
    def build_hybrid_lstm(self, name="Hybrid_LSTM"):
        """Hybrid model combining bidirectional and standard LSTM"""
        model = Sequential([
            Bidirectional(LSTM(units=64, return_sequences=True), input_shape=self.input_shape),
            Dropout(0.2),
            LSTM(units=128, return_sequences=True),
            Dropout(0.2),
            LSTM(units=64),
            Dropout(0.2),
            Dense(units=32, activation='relu'),
            Dropout(0.1),
            Dense(units=1, activation='linear')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mean_squared_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError()]
        )
        
        self.models[name] = model
        return model
    
    def build_all_models(self):
        """Build all model architectures"""
        print("Building multiple LSTM architectures...")
        
        self.build_standard_lstm()
        self.build_bidirectional_lstm()
        self.build_deep_lstm()
        self.build_wide_lstm()
        self.build_hybrid_lstm()
        
        print(f"Built {len(self.models)} different LSTM architectures:")
        for name in self.models.keys():
            print(f"  - {name}")
        
        return self.models
    
    def get_model_summary(self, model_name):
        """Get summary of a specific model"""
        if model_name in self.models:
            print(f"\n{model_name} Architecture:")
            self.models[model_name].summary()
        else:
            print(f"Model {model_name} not found")

print("MultiLSTMModels class defined successfully!")


MultiLSTMModels class defined successfully!


## 4. Enhanced Feature Engineering and Data Preparation


In [8]:
def fetch_stock_data(ticker, start_date, end_date):
    """Enhanced stock data fetching with error handling"""
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date, interval='1d')
        
        if data.empty:
            raise ValueError(f"No data found for ticker {ticker}")
        
        print(f"Successfully fetched {len(data)} days of data for {ticker}")
        print(f"Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
        
        return data
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None

def calculate_enhanced_technical_indicators(data):
    """
    Calculate comprehensive technical indicators
    """
    df = data.copy()
    
    # Moving Averages
    df['SMA_7'] = df['Close'].rolling(window=7).mean()
    df['SMA_21'] = df['Close'].rolling(window=21).mean()
    df['EMA_12'] = df['Close'].ewm(span=12).mean()
    df['EMA_26'] = df['Close'].ewm(span=26).mean()
    
    # MACD
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # RSI
    def calculate_rsi(prices, window=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    df['RSI'] = calculate_rsi(df['Close'])
    df['RSI_14'] = calculate_rsi(df['Close'], 14)
    df['RSI_21'] = calculate_rsi(df['Close'], 21)
    
    # Bollinger Bands
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    bb_std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # Price-based indicators
    df['Price_Change_Pct'] = df['Close'].pct_change()
    df['Price_Change_1d'] = df['Close'].diff()
    df['Price_Change_3d'] = df['Close'].diff(3)
    df['Price_Change_7d'] = df['Close'].diff(7)
    
    # Volume indicators
    df['Volume_MA_7'] = df['Volume'].rolling(window=7).mean()
    df['Volume_MA_21'] = df['Volume'].rolling(window=21).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Volume_MA_21']
    
    # Volatility indicators
    df['HL_Spread'] = (df['High'] - df['Low']) / df['Close']
    df['True_Range'] = np.maximum(
        df['High'] - df['Low'],
        np.maximum(
            abs(df['High'] - df['Close'].shift(1)),
            abs(df['Low'] - df['Close'].shift(1))
        )
    )
    df['ATR'] = df['True_Range'].rolling(window=14).mean()
    
    # Momentum indicators
    df['Momentum_5'] = df['Close'] / df['Close'].shift(5)
    df['Momentum_10'] = df['Close'] / df['Close'].shift(10)
    df['ROC_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)) * 100
    df['ROC_10'] = ((df['Close'] - df['Close'].shift(10)) / df['Close'].shift(10)) * 100
    
    # Support and Resistance levels
    df['High_20'] = df['High'].rolling(window=20).max()
    df['Low_20'] = df['Low'].rolling(window=20).min()
    
    return df

def calculate_dynamic_date_range(daily_sentiment_df):
    """Calculate dynamic date range based on news data"""
    if daily_sentiment_df is None or daily_sentiment_df.empty:
        print("No sentiment data available for date range calculation")
        return None, None
    
    earliest_news_date = daily_sentiment_df.index.min()
    latest_news_date = daily_sentiment_df.index.max()
    start_date = earliest_news_date - pd.DateOffset(years=1)
    
    print(f"\n📅 Dynamic Date Range Calculation:")
    print(f"Earliest news date: {earliest_news_date}")
    print(f"Latest news date: {latest_news_date}")
    print(f"LSTM START_DATE (1 year before): {start_date.strftime('%Y-%m-%d')}")
    print(f"LSTM END_DATE: {latest_news_date.strftime('%Y-%m-%d')}")
    
    return start_date.strftime('%Y-%m-%d'), latest_news_date.strftime('%Y-%m-%d')

# Calculate date range and fetch stock data
START_DATE, END_DATE = calculate_dynamic_date_range(daily_sentiment_df)

if START_DATE and END_DATE:
    print(f"\n📈 Fetching enhanced stock data for {TARGET_STOCK}...")
    stock_data = fetch_stock_data(TARGET_STOCK, START_DATE, END_DATE)
    
    if stock_data is not None:
        enhanced_data = calculate_enhanced_technical_indicators(stock_data)
        enhanced_data = enhanced_data.dropna()
        
        print(f"Enhanced dataset shape: {enhanced_data.shape}")
        print(f"Technical indicators calculated: {len([col for col in enhanced_data.columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume']])}")



📅 Dynamic Date Range Calculation:
Earliest news date: 2011-03-03
Latest news date: 2020-06-10
LSTM START_DATE (1 year before): 2010-03-03
LSTM END_DATE: 2020-06-10

📈 Fetching enhanced stock data for NVDA...
Successfully fetched 2586 days of data for NVDA
Date range: 2010-03-03 to 2020-06-09
Enhanced dataset shape: (2566, 38)
Technical indicators calculated: 33


In [9]:
def create_comprehensive_dataset(stock_data, daily_sentiment_df):
    """
    Create comprehensive dataset merging stock data with enhanced sentiment
    """
    print("\n🔗 Creating comprehensive dataset...")
    
    if daily_sentiment_df is None or daily_sentiment_df.empty:
        print("No sentiment data available for enhancement")
        return stock_data
    
    # Prepare stock data
    comprehensive_data = stock_data.copy()
    comprehensive_data['Date'] = comprehensive_data.index.date
    comprehensive_data = comprehensive_data.set_index('Date')
    
    # Merge with sentiment data
    merged_data = comprehensive_data.join(daily_sentiment_df, how='left')
    
    # Enhanced sentiment feature handling
    sentiment_columns = [
        'Ensemble_Mean', 'Ensemble_Std', 'Ensemble_Min', 'Ensemble_Max',
        'FinBERT_Mean', 'FinBERT_Std', 'VADER_Mean', 'VADER_Std',
        'TextBlob_Mean', 'TextBlob_Std', 'News_Count', 'Positive_Count',
        'Negative_Count', 'Neutral_Count', 'Sentiment_Ratio', 'Sentiment_Volatility'
    ]
    
    for col in sentiment_columns:
        if col in merged_data.columns:
            # Forward fill, then backward fill, then fill with neutral values
            if 'Score' in col or 'Mean' in col or 'Ratio' in col:
                merged_data[col] = merged_data[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
            elif 'Count' in col:
                merged_data[col] = merged_data[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
            elif 'Std' in col or 'Volatility' in col:
                merged_data[col] = merged_data[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
            else:
                merged_data[col] = merged_data[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    # Reset index to datetime
    merged_data.index = pd.to_datetime(merged_data.index)
    
    print(f"Comprehensive dataset shape: {merged_data.shape}")
    print(f"Available sentiment features: {len([col for col in sentiment_columns if col in merged_data.columns])}")
    
    return merged_data

# Create comprehensive dataset
if 'enhanced_data' in locals() and enhanced_data is not None:
    comprehensive_data = create_comprehensive_dataset(enhanced_data, daily_sentiment_df)
    print("\nSample comprehensive data:")
    sample_cols = ['Close', 'SMA_7', 'RSI', 'MACD', 'Ensemble_Mean', 'News_Count', 'Sentiment_Ratio']
    available_sample_cols = [col for col in sample_cols if col in comprehensive_data.columns]
    print(comprehensive_data[available_sample_cols].head(10))



🔗 Creating comprehensive dataset...
Comprehensive dataset shape: (2566, 54)
Available sentiment features: 16

Sample comprehensive data:
               Close     SMA_7        RSI      MACD  Ensemble_Mean  \
Date                                                                 
2010-03-31  0.398868  0.400996  52.799961  0.000836        -0.2316   
2010-04-01  0.394742  0.398802  49.612464  0.000377        -0.2316   
2010-04-05  0.400702  0.399424  53.960482  0.000422        -0.2316   
2010-04-06  0.390845  0.398475  40.826936 -0.000226        -0.2316   
2010-04-07  0.393366  0.397885  37.087836 -0.000553        -0.2316   
2010-04-08  0.386947  0.395658  41.158564 -0.001251        -0.2316   
2010-04-09  0.389469  0.393563  45.912064 -0.001601        -0.2316   
2010-04-12  0.396575  0.393235  46.417331 -0.001348        -0.2316   
2010-04-13  0.404828  0.394676  46.417439 -0.000539        -0.2316   
2010-04-14  0.409871  0.395986  60.424159  0.000464        -0.2316   

            News_Coun

In [10]:
class EnhancedDataPreparation:
    """
    Enhanced data preparation for multiple model training
    """
    
    def __init__(self, sequence_length=30, test_size=0.2):
        self.sequence_length = sequence_length
        self.test_size = test_size
        self.scalers = {}
        self.feature_sets = {}
        
    def prepare_baseline_features(self, data):
        """Prepare baseline technical features"""
        baseline_features = [
            'Close', 'SMA_7', 'SMA_21', 'RSI', 'MACD', 'Volume', 
            'HL_Spread', 'ATR', 'BB_Position', 'Volume_Ratio'
        ]
        available_features = [col for col in baseline_features if col in data.columns]
        print(f"Baseline features ({len(available_features)}): {available_features}")
        return available_features
    
    def prepare_sentiment_features(self, data):
        """Prepare sentiment-enhanced features"""
        baseline_features = self.prepare_baseline_features(data)
        sentiment_features = [
            'Ensemble_Mean', 'Sentiment_Ratio', 'News_Count', 
            'Sentiment_Volatility', 'FinBERT_Mean', 'VADER_Mean'
        ]
        available_sentiment = [col for col in sentiment_features if col in data.columns]
        all_features = baseline_features + available_sentiment
        print(f"Sentiment-enhanced features ({len(all_features)}): {all_features}")
        return all_features
    
    def prepare_comprehensive_features(self, data):
        """Prepare comprehensive feature set"""
        comprehensive_features = [
            'Close', 'SMA_7', 'SMA_21', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal',
            'RSI', 'RSI_14', 'RSI_21', 'BB_Position', 'BB_Width', 'Volume', 
            'Volume_Ratio', 'HL_Spread', 'ATR', 'Momentum_5', 'ROC_5',
            'Ensemble_Mean', 'Ensemble_Std', 'Sentiment_Ratio', 'News_Count',
            'Sentiment_Volatility', 'FinBERT_Mean', 'VADER_Mean', 'TextBlob_Mean'
        ]
        available_features = [col for col in comprehensive_features if col in data.columns]
        print(f"Comprehensive features ({len(available_features)}): {available_features}")
        return available_features
    
    def create_sequences(self, data, feature_columns, target_column='Close'):
        """Create sequences for LSTM training"""
        features = data[feature_columns].values
        
        # Scale features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_features = scaler.fit_transform(features)
        
        # Create sequences
        X, y = [], []
        for i in range(self.sequence_length, len(scaled_features)):
            X.append(scaled_features[i-self.sequence_length:i])
            y.append(scaled_features[i, feature_columns.index(target_column)])
        
        X, y = np.array(X), np.array(y)
        
        # Split data
        split_index = int(len(X) * (1 - self.test_size))
        X_train, X_test = X[:split_index], X[split_index:]
        y_train, y_test = y[:split_index], y[split_index:]
        
        return X_train, X_test, y_train, y_test, scaler
    
    def prepare_all_datasets(self, data):
        """Prepare all feature sets for comparison"""
        datasets = {}
        
        # Baseline dataset
        baseline_features = self.prepare_baseline_features(data)
        if baseline_features:
            X_train, X_test, y_train, y_test, scaler = self.create_sequences(data, baseline_features)
            datasets['baseline'] = {
                'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test,
                'scaler': scaler, 'features': baseline_features
            }
        
        # Sentiment-enhanced dataset
        sentiment_features = self.prepare_sentiment_features(data)
        if sentiment_features:
            X_train, X_test, y_train, y_test, scaler = self.create_sequences(data, sentiment_features)
            datasets['sentiment'] = {
                'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test,
                'scaler': scaler, 'features': sentiment_features
            }
        
        # Comprehensive dataset
        comprehensive_features = self.prepare_comprehensive_features(data)
        if comprehensive_features:
            X_train, X_test, y_train, y_test, scaler = self.create_sequences(data, comprehensive_features)
            datasets['comprehensive'] = {
                'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test,
                'scaler': scaler, 'features': comprehensive_features
            }
        
        print(f"\nPrepared {len(datasets)} different datasets:")
        for name, dataset in datasets.items():
            print(f"  {name}: {dataset['X_train'].shape} -> {dataset['y_train'].shape}")
        
        return datasets

# Initialize data preparation
if 'comprehensive_data' in locals():
    data_prep = EnhancedDataPreparation(sequence_length=30, test_size=0.2)
    all_datasets = data_prep.prepare_all_datasets(comprehensive_data)
    print("Enhanced data preparation completed!")


Baseline features (10): ['Close', 'SMA_7', 'SMA_21', 'RSI', 'MACD', 'Volume', 'HL_Spread', 'ATR', 'BB_Position', 'Volume_Ratio']
Baseline features (10): ['Close', 'SMA_7', 'SMA_21', 'RSI', 'MACD', 'Volume', 'HL_Spread', 'ATR', 'BB_Position', 'Volume_Ratio']
Sentiment-enhanced features (16): ['Close', 'SMA_7', 'SMA_21', 'RSI', 'MACD', 'Volume', 'HL_Spread', 'ATR', 'BB_Position', 'Volume_Ratio', 'Ensemble_Mean', 'Sentiment_Ratio', 'News_Count', 'Sentiment_Volatility', 'FinBERT_Mean', 'VADER_Mean']
Comprehensive features (26): ['Close', 'SMA_7', 'SMA_21', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'RSI', 'RSI_14', 'RSI_21', 'BB_Position', 'BB_Width', 'Volume', 'Volume_Ratio', 'HL_Spread', 'ATR', 'Momentum_5', 'ROC_5', 'Ensemble_Mean', 'Ensemble_Std', 'Sentiment_Ratio', 'News_Count', 'Sentiment_Volatility', 'FinBERT_Mean', 'VADER_Mean', 'TextBlob_Mean']

Prepared 3 different datasets:
  baseline: (2028, 30, 10) -> (2028,)
  sentiment: (2028, 30, 16) -> (2028,)
  comprehensive: (2028, 30, 2

## 5. Comprehensive Training and Evaluation System


In [12]:
class ModelTrainingPipeline:
    """
    Comprehensive pipeline for training multiple models with different configurations
    """
    
    def __init__(self, datasets, target_stock):
        self.datasets = datasets
        self.target_stock = target_stock
        self.results = {}
        self.trained_models = {}
        self.epoch_configs = [25, 50, 75, 100]
        
    def calculate_comprehensive_metrics(self, actual, predicted, model_name, dataset_name, epochs):
        """Calculate comprehensive performance metrics"""
        mse = mean_squared_error(actual, predicted)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        
        # Mean Absolute Percentage Error
        mape = np.mean(np.abs((actual - predicted) / actual)) * 100
        
        # Directional Accuracy
        actual_direction = np.diff(actual.flatten()) > 0
        predicted_direction = np.diff(predicted.flatten()) > 0
        directional_accuracy = np.mean(actual_direction == predicted_direction) * 100
        
        # Theil's U statistic
        naive_forecast = np.roll(actual.flatten(), 1)[1:]
        actual_changes = actual.flatten()[1:]
        predicted_changes = predicted.flatten()[1:]
        
        theil_u = np.sqrt(np.mean((predicted_changes - actual_changes)**2)) / \
                  np.sqrt(np.mean((naive_forecast - actual_changes)**2))
        
        return {
            'Model': model_name,
            'Dataset': dataset_name,
            'Epochs': epochs,
            'RMSE': rmse,
            'MAE': mae,
            'MAPE': mape,
            'R2_Score': r2,
            'Directional_Accuracy': directional_accuracy,
            'Theil_U': theil_u
        }
    
    def train_single_model(self, model_name, model, dataset_name, dataset, epochs):
        """Train a single model configuration"""
        print(f"\n🚀 Training {model_name} on {dataset_name} dataset for {epochs} epochs...")
        
        # Prepare callbacks
        callbacks = [
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=0),
            EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
        ]
        
        # Train model
        history = model.fit(
            dataset['X_train'], dataset['y_train'],
            epochs=epochs,
            batch_size=32,
            validation_split=0.2,
            callbacks=callbacks,
            verbose=0
        )
        
        # Make predictions
        train_predictions = model.predict(dataset['X_train'], verbose=0)
        test_predictions = model.predict(dataset['X_test'], verbose=0)
        
        # Create scaler for inverse transformation (Close price only)
        close_scaler = MinMaxScaler(feature_range=(0, 1))
        close_data = comprehensive_data[['Close']].values
        close_scaler.fit(close_data)
        
        # Inverse transform
        train_actual = close_scaler.inverse_transform(dataset['y_train'].reshape(-1, 1))
        test_actual = close_scaler.inverse_transform(dataset['y_test'].reshape(-1, 1))
        train_pred_scaled = close_scaler.inverse_transform(train_predictions)
        test_pred_scaled = close_scaler.inverse_transform(test_predictions)
        
        # Calculate metrics
        train_metrics = self.calculate_comprehensive_metrics(
            train_actual, train_pred_scaled, model_name, dataset_name, epochs
        )
        test_metrics = self.calculate_comprehensive_metrics(
            test_actual, test_pred_scaled, model_name, dataset_name, epochs
        )
        
        # Store results
        result_key = f"{model_name}_{dataset_name}_{epochs}epochs"
        self.results[result_key] = {
            'train_metrics': train_metrics,
            'test_metrics': test_metrics,
            'history': history,
            'predictions': {
                'train_actual': train_actual,
                'train_predicted': train_pred_scaled,
                'test_actual': test_actual,
                'test_predicted': test_pred_scaled
            }
        }
        
        self.trained_models[result_key] = model
        
        print(f"✅ {model_name} completed - Test RMSE: ${test_metrics['RMSE']:.2f}, "
              f"Directional Accuracy: {test_metrics['Directional_Accuracy']:.1f}%")
        
        return test_metrics
    
    def train_all_configurations(self):
        """Train all model and dataset combinations with different epochs"""
        print(f"\n🎯 Starting comprehensive training for {self.target_stock}")
        print(f"Models: 5 architectures")
        print(f"Datasets: {len(self.datasets)} feature sets")
        print(f"Epochs: {self.epoch_configs}")
        print(f"Total configurations: {5 * len(self.datasets) * len(self.epoch_configs)}")
        
        all_test_metrics = []
        
        for dataset_name, dataset in self.datasets.items():
            print(f"\n📊 Processing {dataset_name} dataset...")
            
            # Initialize models for this dataset
            input_shape = (dataset['X_train'].shape[1], dataset['X_train'].shape[2])
            model_builder = MultiLSTMModels(input_shape)
            models = model_builder.build_all_models()
            
            for model_name, model in models.items():
                for epochs in self.epoch_configs:
                    # Create a fresh copy of the model for each training
                    fresh_model = tf.keras.models.clone_model(model)
                    fresh_model.compile(
                        optimizer=model.optimizer,
                        loss=model.loss,
                        metrics=[tf.keras.metrics.MeanAbsoluteError()]
                    )
                    
                    test_metrics = self.train_single_model(
                        model_name, fresh_model, dataset_name, dataset, epochs
                    )
                    all_test_metrics.append(test_metrics)
        
        print(f"\n✅ Training completed! Total configurations trained: {len(all_test_metrics)}")
        return all_test_metrics
    
    def get_best_configurations(self, metric='RMSE', top_n=10):
        """Get best performing configurations"""
        all_metrics = []
        for result in self.results.values():
            all_metrics.append(result['test_metrics'])
        
        df = pd.DataFrame(all_metrics)
        
        if metric in ['RMSE', 'MAE', 'MAPE', 'Theil_U']:
            # Lower is better
            best_configs = df.nsmallest(top_n, metric)
        else:
            # Higher is better (R2_Score, Directional_Accuracy)
            best_configs = df.nlargest(top_n, metric)
        
        return best_configs
    
    def create_performance_summary(self):
        """Create comprehensive performance summary"""
        all_metrics = []
        for result in self.results.values():
            all_metrics.append(result['test_metrics'])
        
        df = pd.DataFrame(all_metrics)
        
        print(f"\n📈 COMPREHENSIVE PERFORMANCE SUMMARY - {self.target_stock}")
        print("=" * 80)
        
        # Overall statistics
        print(f"\nTotal configurations tested: {len(df)}")
        print(f"Models: {df['Model'].nunique()}")
        print(f"Datasets: {df['Dataset'].nunique()}")
        print(f"Epoch configurations: {sorted(df['Epochs'].unique())}")
        
        # Best performers by metric
        print(f"\n🏆 TOP PERFORMERS BY METRIC:")
        print("-" * 50)
        
        metrics_to_show = ['RMSE', 'MAE', 'Directional_Accuracy', 'R2_Score']
        for metric in metrics_to_show:
            if metric in ['RMSE', 'MAE']:
                best = df.loc[df[metric].idxmin()]
                print(f"{metric:20s}: {best[metric]:.4f} ({best['Model']} - {best['Dataset']} - {best['Epochs']} epochs)")
            else:
                best = df.loc[df[metric].idxmax()]
                print(f"{metric:20s}: {best[metric]:.4f} ({best['Model']} - {best['Dataset']} - {best['Epochs']} epochs)")
        
        # Performance by model architecture
        print(f"\n📊 AVERAGE PERFORMANCE BY MODEL ARCHITECTURE:")
        print("-" * 60)
        model_performance = df.groupby('Model')[['RMSE', 'MAE', 'Directional_Accuracy', 'R2_Score']].mean()
        print(model_performance.round(4))
        
        # Performance by dataset
        print(f"\n📊 AVERAGE PERFORMANCE BY DATASET:")
        print("-" * 45)
        dataset_performance = df.groupby('Dataset')[['RMSE', 'MAE', 'Directional_Accuracy', 'R2_Score']].mean()
        print(dataset_performance.round(4))
        
        # Performance by epochs
        print(f"\n📊 AVERAGE PERFORMANCE BY EPOCHS:")
        print("-" * 40)
        epoch_performance = df.groupby('Epochs')[['RMSE', 'MAE', 'Directional_Accuracy', 'R2_Score']].mean()
        print(epoch_performance.round(4))
        
        return df

# Initialize training pipeline
if 'all_datasets' in locals() and all_datasets:
    print("Initializing comprehensive training pipeline...")
    training_pipeline = ModelTrainingPipeline(all_datasets, TARGET_STOCK)
    
    # Start comprehensive training
    all_test_metrics = training_pipeline.train_all_configurations()
    
    # Create performance summary
    performance_df = training_pipeline.create_performance_summary()
    
    print("\n🎉 Comprehensive training and evaluation completed!")


Initializing comprehensive training pipeline...

🎯 Starting comprehensive training for NVDA
Models: 5 architectures
Datasets: 3 feature sets
Epochs: [25, 50, 75, 100]
Total configurations: 60

📊 Processing baseline dataset...
Building multiple LSTM architectures...
Built 5 different LSTM architectures:
  - Standard_LSTM
  - Bidirectional_LSTM
  - Deep_LSTM
  - Wide_LSTM
  - Hybrid_LSTM

🚀 Training Standard_LSTM on baseline dataset for 25 epochs...
✅ Standard_LSTM completed - Test RMSE: $0.70, Directional Accuracy: 50.7%

🚀 Training Standard_LSTM on baseline dataset for 50 epochs...


NotImplementedError: numpy() is only available when eager execution is enabled.

In [ ]:
# Display top 10 best configurations
if 'training_pipeline' in locals():
    print("\n🏆 TOP 10 BEST CONFIGURATIONS (by RMSE):")
    print("=" * 80)
    best_rmse = training_pipeline.get_best_configurations('RMSE', 10)
    print(best_rmse.to_string(index=False))
    
    print("\n🏆 TOP 10 BEST CONFIGURATIONS (by Directional Accuracy):")
    print("=" * 80)
    best_directional = training_pipeline.get_best_configurations('Directional_Accuracy', 10)
    print(best_directional.to_string(index=False))


## 6. Advanced Visualization and Analysis


In [ ]:
class AdvancedVisualization:
    """
    Comprehensive visualization system for model analysis
    """
    
    def __init__(self, training_pipeline, company_sentiment_df, target_stock):
        self.pipeline = training_pipeline
        self.sentiment_df = company_sentiment_df
        self.target_stock = target_stock
        self.performance_df = None
        
        # Set up plotting style
        plt.style.use('seaborn-v0_8')
        sns.set_palette("husl")
        
    def create_sentiment_analysis_dashboard(self):
        """Create comprehensive sentiment analysis dashboard"""
        if self.sentiment_df is None or self.sentiment_df.empty:
            print("No sentiment data available for visualization")
            return
        
        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        fig.suptitle(f'{self.target_stock} - Comprehensive Sentiment Analysis Dashboard', 
                     fontsize=16, fontweight='bold')
        
        # 1. Sentiment distribution comparison
        models = ['Ensemble', 'FinBERT', 'VADER', 'TextBlob']
        sentiment_counts = {}
        
        for model in models:
            label_col = f'{model}_Label'
            if label_col in self.sentiment_df.columns:
                counts = self.sentiment_df[label_col].value_counts()
                sentiment_counts[model] = counts
        
        if sentiment_counts:
            sentiment_df_plot = pd.DataFrame(sentiment_counts).fillna(0)
            sentiment_df_plot.plot(kind='bar', ax=axes[0, 0], width=0.8)
            axes[0, 0].set_title('Sentiment Distribution by Model')
            axes[0, 0].set_xlabel('Sentiment')
            axes[0, 0].set_ylabel('Count')
            axes[0, 0].legend(title='Models')
            axes[0, 0].tick_params(axis='x', rotation=45)
        
        # 2. Sentiment scores comparison
        score_cols = [col for col in self.sentiment_df.columns if 'Score' in col]
        if score_cols:
            self.sentiment_df[score_cols].plot(kind='box', ax=axes[0, 1])
            axes[0, 1].set_title('Sentiment Score Distributions')
            axes[0, 1].set_ylabel('Sentiment Score')
            axes[0, 1].tick_params(axis='x', rotation=45)
        
        # 3. Daily sentiment timeline
        daily_sentiment = self.sentiment_df.groupby('Date').agg({
            'Ensemble_Score': 'mean',
            'FinBERT_Score': 'mean',
            'VADER_Score': 'mean',
            'TextBlob_Score': 'mean'
        }).fillna(0)
        
        if not daily_sentiment.empty:
            daily_sentiment.plot(ax=axes[0, 2], alpha=0.7)
            axes[0, 2].set_title('Daily Average Sentiment Scores')
            axes[0, 2].set_xlabel('Date')
            axes[0, 2].set_ylabel('Average Sentiment Score')
            axes[0, 2].legend(title='Models')
            axes[0, 2].tick_params(axis='x', rotation=45)
        
        # 4. Sentiment correlation heatmap
        score_data = self.sentiment_df[score_cols] if score_cols else pd.DataFrame()
        if not score_data.empty:
            correlation_matrix = score_data.corr()
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                       ax=axes[1, 0], square=True)
            axes[1, 0].set_title('Sentiment Model Correlations')
        
        # 5. News volume over time
        news_volume = self.sentiment_df.groupby('Date').size()
        if not news_volume.empty:
            news_volume.plot(kind='line', ax=axes[1, 1], color='green', alpha=0.7)
            axes[1, 1].set_title('Daily News Volume')
            axes[1, 1].set_xlabel('Date')
            axes[1, 1].set_ylabel('Number of Articles')
            axes[1, 1].tick_params(axis='x', rotation=45)
        
        # 6. Sentiment vs News Volume
        if not daily_sentiment.empty and not news_volume.empty:
            combined_data = pd.concat([daily_sentiment['Ensemble_Score'], news_volume], axis=1)
            combined_data.columns = ['Sentiment', 'Volume']
            combined_data = combined_data.dropna()
            
            if not combined_data.empty:
                axes[1, 2].scatter(combined_data['Volume'], combined_data['Sentiment'], 
                                 alpha=0.6, color='purple')
                axes[1, 2].set_title('Sentiment vs News Volume')
                axes[1, 2].set_xlabel('Daily News Volume')
                axes[1, 2].set_ylabel('Average Sentiment Score')
                
                # Add trend line
                z = np.polyfit(combined_data['Volume'], combined_data['Sentiment'], 1)
                p = np.poly1d(z)
                axes[1, 2].plot(combined_data['Volume'], p(combined_data['Volume']), 
                              "r--", alpha=0.8)
        
        plt.tight_layout()
        plt.show()
    
    def create_model_performance_heatmap(self):
        """Create performance heatmap for all model configurations"""
        if not self.pipeline.results:
            print("No training results available for heatmap")
            return
        
        # Prepare data for heatmap
        all_metrics = []
        for result in self.pipeline.results.values():
            all_metrics.append(result['test_metrics'])
        
        df = pd.DataFrame(all_metrics)
        
        # Create pivot tables for different metrics
        metrics = ['RMSE', 'MAE', 'Directional_Accuracy', 'R2_Score']
        
        fig, axes = plt.subplots(2, 2, figsize=(20, 16))
        fig.suptitle(f'{self.target_stock} - Model Performance Heatmaps', 
                     fontsize=16, fontweight='bold')
        
        axes = axes.flatten()
        
        for i, metric in enumerate(metrics):
            # Create pivot table: Models vs Datasets, averaged across epochs
            pivot_data = df.groupby(['Model', 'Dataset'])[metric].mean().unstack()
            
            # Create heatmap
            if metric in ['RMSE', 'MAE']:
                cmap = 'Reds_r'  # Lower is better, so reverse colormap
            else:
                cmap = 'Greens'  # Higher is better
            
            sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap=cmap, 
                       ax=axes[i], square=True, cbar_kws={'label': metric})
            axes[i].set_title(f'{metric} by Model and Dataset')
            axes[i].set_xlabel('Dataset')
            axes[i].set_ylabel('Model')
        
        plt.tight_layout()
        plt.show()
    
    def create_epoch_analysis(self):
        """Analyze performance across different epoch configurations"""
        if not self.pipeline.results:
            print("No training results available for epoch analysis")
            return
        
        all_metrics = []
        for result in self.pipeline.results.values():
            all_metrics.append(result['test_metrics'])
        
        df = pd.DataFrame(all_metrics)
        
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        fig.suptitle(f'{self.target_stock} - Epoch Analysis', fontsize=16, fontweight='bold')
        
        # 1. RMSE vs Epochs
        epoch_rmse = df.groupby(['Epochs', 'Model'])['RMSE'].mean().unstack()
        epoch_rmse.plot(ax=axes[0, 0], marker='o')
        axes[0, 0].set_title('RMSE vs Epochs by Model')
        axes[0, 0].set_xlabel('Epochs')
        axes[0, 0].set_ylabel('RMSE')
        axes[0, 0].legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Directional Accuracy vs Epochs
        epoch_da = df.groupby(['Epochs', 'Model'])['Directional_Accuracy'].mean().unstack()
        epoch_da.plot(ax=axes[0, 1], marker='s')
        axes[0, 1].set_title('Directional Accuracy vs Epochs by Model')
        axes[0, 1].set_xlabel('Epochs')
        axes[0, 1].set_ylabel('Directional Accuracy (%)')
        axes[0, 1].legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Performance by Dataset and Epochs
        dataset_epoch = df.groupby(['Epochs', 'Dataset'])['RMSE'].mean().unstack()
        dataset_epoch.plot(kind='bar', ax=axes[1, 0], width=0.8)
        axes[1, 0].set_title('RMSE by Dataset and Epochs')
        axes[1, 0].set_xlabel('Epochs')
        axes[1, 0].set_ylabel('RMSE')
        axes[1, 0].legend(title='Dataset')
        axes[1, 0].tick_params(axis='x', rotation=0)
        
        # 4. Optimal epochs distribution
        best_epochs = df.loc[df.groupby(['Model', 'Dataset'])['RMSE'].idxmin(), 'Epochs']
        best_epochs.value_counts().plot(kind='bar', ax=axes[1, 1], color='skyblue')
        axes[1, 1].set_title('Distribution of Optimal Epochs')
        axes[1, 1].set_xlabel('Epochs')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].tick_params(axis='x', rotation=0)
        
        plt.tight_layout()
        plt.show()
    
    def create_prediction_comparison(self, top_n=3):
        """Create prediction comparison for top performing models"""
        if not self.pipeline.results:
            print("No training results available for prediction comparison")
            return
        
        # Get top N models by RMSE
        all_metrics = []
        for key, result in self.pipeline.results.items():
            metrics = result['test_metrics'].copy()
            metrics['key'] = key
            all_metrics.append(metrics)
        
        df = pd.DataFrame(all_metrics)
        top_models = df.nsmallest(top_n, 'RMSE')
        
        fig, axes = plt.subplots(top_n, 2, figsize=(20, 6*top_n))
        if top_n == 1:
            axes = axes.reshape(1, -1)
        
        fig.suptitle(f'{self.target_stock} - Top {top_n} Model Predictions Comparison', 
                     fontsize=16, fontweight='bold')
        
        for i, (_, model_info) in enumerate(top_models.iterrows()):
            key = model_info['key']
            result = self.pipeline.results[key]
            predictions = result['predictions']
            
            # Create date index for test data
            test_size = len(predictions['test_actual'])
            dates = pd.date_range(end=comprehensive_data.index[-1], periods=test_size, freq='D')
            
            # Plot predictions vs actual
            axes[i, 0].plot(dates, predictions['test_actual'], 
                           label='Actual', color='blue', linewidth=2)
            axes[i, 0].plot(dates, predictions['test_predicted'], 
                           label='Predicted', color='red', linewidth=2, alpha=0.8)
            axes[i, 0].set_title(f'{model_info["Model"]} - {model_info["Dataset"]} - {model_info["Epochs"]} epochs\n'
                               f'RMSE: ${model_info["RMSE"]:.2f}, DA: {model_info["Directional_Accuracy"]:.1f}%')
            axes[i, 0].set_ylabel('Price ($)')
            axes[i, 0].legend()
            axes[i, 0].grid(True, alpha=0.3)
            axes[i, 0].tick_params(axis='x', rotation=45)
            
            # Plot prediction errors
            errors = predictions['test_actual'].flatten() - predictions['test_predicted'].flatten()
            axes[i, 1].hist(errors, bins=30, alpha=0.7, color='green', edgecolor='black')
            axes[i, 1].axvline(np.mean(errors), color='red', linestyle='--', 
                             label=f'Mean Error: ${np.mean(errors):.2f}')
            axes[i, 1].set_title('Prediction Error Distribution')
            axes[i, 1].set_xlabel('Prediction Error ($)')
            axes[i, 1].set_ylabel('Frequency')
            axes[i, 1].legend()
            axes[i, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def create_comprehensive_dashboard(self):
        """Create comprehensive analysis dashboard"""
        print(f"\n📊 Creating comprehensive visualization dashboard for {self.target_stock}...")
        
        # 1. Sentiment Analysis Dashboard
        print("Creating sentiment analysis dashboard...")
        self.create_sentiment_analysis_dashboard()
        
        # 2. Model Performance Heatmaps
        print("Creating model performance heatmaps...")
        self.create_model_performance_heatmap()
        
        # 3. Epoch Analysis
        print("Creating epoch analysis...")
        self.create_epoch_analysis()
        
        # 4. Top Model Predictions
        print("Creating prediction comparisons...")
        self.create_prediction_comparison(top_n=3)
        
        print("✅ Comprehensive visualization dashboard completed!")

# Create comprehensive visualizations
if 'training_pipeline' in locals() and 'company_sentiment_df' in locals():
    print("Initializing advanced visualization system...")
    viz = AdvancedVisualization(training_pipeline, company_sentiment_df, TARGET_STOCK)
    viz.create_comprehensive_dashboard()


## 7. Final Comprehensive Report and Summary


In [ ]:
def generate_final_report(training_pipeline, target_stock):
    """
    Generate comprehensive final report with all findings
    """
    print("=" * 100)
    print(f"🎯 FINAL COMPREHENSIVE ANALYSIS REPORT - {target_stock}")
    print("=" * 100)
    
    # Get performance data
    all_metrics = []
    for result in training_pipeline.results.values():
        all_metrics.append(result['test_metrics'])
    
    df = pd.DataFrame(all_metrics)
    
    print(f"\n📊 EXECUTIVE SUMMARY")
    print("-" * 50)
    print(f"Target Stock: {target_stock}")
    print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total Model Configurations Tested: {len(df)}")
    print(f"Model Architectures: {df['Model'].nunique()} (Standard, Bidirectional, Deep, Wide, Hybrid)")
    print(f"Feature Sets: {df['Dataset'].nunique()} (Baseline, Sentiment-Enhanced, Comprehensive)")
    print(f"Epoch Configurations: {len(df['Epochs'].unique())} ({sorted(df['Epochs'].unique())})")
    
    # Best overall model
    best_model = df.loc[df['RMSE'].idxmin()]
    print(f"\n🏆 BEST OVERALL MODEL")
    print("-" * 30)
    print(f"Architecture: {best_model['Model']}")
    print(f"Feature Set: {best_model['Dataset']}")
    print(f"Optimal Epochs: {best_model['Epochs']}")
    print(f"RMSE: ${best_model['RMSE']:.2f}")
    print(f"MAE: ${best_model['MAE']:.2f}")
    print(f"MAPE: {best_model['MAPE']:.2f}%")
    print(f"Directional Accuracy: {best_model['Directional_Accuracy']:.1f}%")
    print(f"R² Score: {best_model['R2_Score']:.4f}")
    print(f"Theil's U: {best_model['Theil_U']:.4f}")
    
    # Model architecture analysis
    print(f"\n📈 MODEL ARCHITECTURE PERFORMANCE RANKING")
    print("-" * 55)
    model_ranking = df.groupby('Model').agg({
        'RMSE': 'mean',
        'Directional_Accuracy': 'mean',
        'R2_Score': 'mean'
    }).round(4)
    model_ranking = model_ranking.sort_values('RMSE')
    
    for i, (model, metrics) in enumerate(model_ranking.iterrows(), 1):
        print(f"{i}. {model:20s} - RMSE: ${metrics['RMSE']:.2f}, "
              f"DA: {metrics['Directional_Accuracy']:.1f}%, R²: {metrics['R2_Score']:.3f}")
    
    # Feature set analysis
    print(f"\n📊 FEATURE SET EFFECTIVENESS")
    print("-" * 35)
    dataset_ranking = df.groupby('Dataset').agg({
        'RMSE': 'mean',
        'Directional_Accuracy': 'mean',
        'R2_Score': 'mean'
    }).round(4)
    dataset_ranking = dataset_ranking.sort_values('RMSE')
    
    for i, (dataset, metrics) in enumerate(dataset_ranking.iterrows(), 1):
        print(f"{i}. {dataset:15s} - RMSE: ${metrics['RMSE']:.2f}, "
              f"DA: {metrics['Directional_Accuracy']:.1f}%, R²: {metrics['R2_Score']:.3f}")
    
    # Epoch analysis
    print(f"\n⏱️  OPTIMAL EPOCH ANALYSIS")
    print("-" * 30)
    epoch_ranking = df.groupby('Epochs').agg({
        'RMSE': 'mean',
        'Directional_Accuracy': 'mean'
    }).round(4)
    
    best_epoch = epoch_ranking.loc[epoch_ranking['RMSE'].idxmin()]
    print(f"Best Average Performance: {best_epoch.name} epochs")
    print(f"  Average RMSE: ${best_epoch['RMSE']:.2f}")
    print(f"  Average Directional Accuracy: {best_epoch['Directional_Accuracy']:.1f}%")
    
    # Most frequent optimal epochs
    optimal_epochs = df.loc[df.groupby(['Model', 'Dataset'])['RMSE'].idxmin(), 'Epochs']
    most_common_epoch = optimal_epochs.mode()[0]
    print(f"Most Frequently Optimal: {most_common_epoch} epochs ({(optimal_epochs == most_common_epoch).sum()}/{len(optimal_epochs)} configurations)")
    
    # Sentiment analysis impact
    if 'sentiment' in df['Dataset'].values:
        baseline_perf = df[df['Dataset'] == 'baseline']['RMSE'].mean()
        sentiment_perf = df[df['Dataset'] == 'sentiment']['RMSE'].mean()
        improvement = ((baseline_perf - sentiment_perf) / baseline_perf) * 100
        
        print(f"\n💭 SENTIMENT ANALYSIS IMPACT")
        print("-" * 35)
        print(f"Baseline Average RMSE: ${baseline_perf:.2f}")
        print(f"Sentiment-Enhanced Average RMSE: ${sentiment_perf:.2f}")
        print(f"Average Improvement: {improvement:.2f}%")
        
        if improvement > 5:
            print("✅ Sentiment analysis provides SIGNIFICANT improvement")
        elif improvement > 0:
            print("✅ Sentiment analysis provides MODEST improvement")
        else:
            print("⚠️  Sentiment analysis shows MINIMAL impact")
    
    # Key insights and recommendations
    print(f"\n🔍 KEY INSIGHTS AND RECOMMENDATIONS")
    print("-" * 45)
    
    # Best architecture insight
    best_arch = model_ranking.index[0]
    print(f"1. BEST ARCHITECTURE: {best_arch}")
    print(f"   - Consistently outperforms other architectures")
    print(f"   - Average RMSE: ${model_ranking.loc[best_arch, 'RMSE']:.2f}")
    
    # Best feature set insight
    best_features = dataset_ranking.index[0]
    print(f"\n2. OPTIMAL FEATURE SET: {best_features}")
    print(f"   - Provides best balance of accuracy and generalization")
    print(f"   - Average RMSE: ${dataset_ranking.loc[best_features, 'RMSE']:.2f}")
    
    # Epoch recommendation
    print(f"\n3. RECOMMENDED EPOCHS: {most_common_epoch}")
    print(f"   - Most frequently optimal across configurations")
    print(f"   - Balances training time and performance")
    
    # Model complexity insight
    model_complexity = {
        'Standard_LSTM': 'Medium',
        'Bidirectional_LSTM': 'High',
        'Deep_LSTM': 'Very High',
        'Wide_LSTM': 'High',
        'Hybrid_LSTM': 'Very High'
    }
    
    print(f"\n4. COMPLEXITY vs PERFORMANCE:")
    for model in model_ranking.index[:3]:
        complexity = model_complexity.get(model, 'Unknown')
        rmse = model_ranking.loc[model, 'RMSE']
        print(f"   - {model}: {complexity} complexity, RMSE: ${rmse:.2f}")
    
    print(f"\n📋 IMPLEMENTATION RECOMMENDATIONS")
    print("-" * 40)
    print(f"For production deployment of {target_stock} price prediction:")
    print(f"1. Use {best_model['Model']} architecture")
    print(f"2. Include {best_model['Dataset']} features")
    print(f"3. Train for {best_model['Epochs']} epochs")
    print(f"4. Expected performance: RMSE ~${best_model['RMSE']:.2f}, DA ~{best_model['Directional_Accuracy']:.1f}%")
    print(f"5. Monitor model performance and retrain monthly")
    
    return df

# Generate final comprehensive report
if 'training_pipeline' in locals():
    print("Generating final comprehensive report...")
    final_report_df = generate_final_report(training_pipeline, TARGET_STOCK)


In [ ]:
def create_improvement_summary():
    """
    Summarize improvements made over the original analysis
    """
    print("\n" + "=" * 80)
    print("🚀 IMPROVEMENTS OVER ORIGINAL ANALYSIS")
    print("=" * 80)
    
    improvements = [
        {
            "Category": "Sentiment Analysis",
            "Original": "Single FinBERT model",
            "Enhanced": "Multiple models (FinBERT, VADER, TextBlob) + Ensemble",
            "Benefit": "More robust and accurate sentiment scoring"
        },
        {
            "Category": "LSTM Architectures", 
            "Original": "Single standard LSTM (3 layers)",
            "Enhanced": "5 different architectures (Standard, Bidirectional, Deep, Wide, Hybrid)",
            "Benefit": "Comprehensive architecture comparison and optimization"
        },
        {
            "Category": "Feature Engineering",
            "Original": "5 basic technical indicators",
            "Enhanced": "25+ advanced technical indicators + sentiment features",
            "Benefit": "Richer feature set for better predictions"
        },
        {
            "Category": "Epoch Optimization",
            "Original": "Fixed 50 epochs",
            "Enhanced": "Dynamic testing (25, 50, 75, 100 epochs)",
            "Benefit": "Optimal training duration for each configuration"
        },
        {
            "Category": "Model Evaluation",
            "Original": "2 model comparison",
            "Enhanced": "60 configuration comparison (5×3×4)",
            "Benefit": "Comprehensive performance analysis"
        },
        {
            "Category": "Metrics",
            "Original": "RMSE, MAE, Directional Accuracy",
            "Enhanced": "RMSE, MAE, MAPE, R², Directional Accuracy, Theil's U",
            "Benefit": "More comprehensive performance assessment"
        },
        {
            "Category": "Visualizations",
            "Original": "Basic 2×2 and 2×3 plots",
            "Enhanced": "Advanced dashboards with heatmaps, correlation analysis",
            "Benefit": "Better insights and decision-making support"
        },
        {
            "Category": "Data Processing",
            "Original": "Basic sentiment aggregation",
            "Enhanced": "Advanced sentiment metrics + volatility analysis",
            "Benefit": "More nuanced sentiment feature engineering"
        }
    ]
    
    for i, improvement in enumerate(improvements, 1):
        print(f"\n{i}. {improvement['Category'].upper()}")
        print(f"   Original: {improvement['Original']}")
        print(f"   Enhanced: {improvement['Enhanced']}")
        print(f"   Benefit:  {improvement['Benefit']}")
    
    print(f"\n📈 QUANTITATIVE IMPROVEMENTS")
    print("-" * 35)
    print(f"• Model configurations tested: 2 → 60 (30x increase)")
    print(f"• Sentiment models: 1 → 4 (4x increase)")
    print(f"• LSTM architectures: 1 → 5 (5x increase)")
    print(f"• Technical indicators: 5 → 25+ (5x increase)")
    print(f"• Performance metrics: 3 → 6 (2x increase)")
    print(f"• Visualization charts: 4 → 15+ (4x increase)")
    
    print(f"\n🎯 BUSINESS VALUE")
    print("-" * 20)
    print(f"• More accurate predictions through ensemble methods")
    print(f"• Better risk assessment with multiple metrics")
    print(f"• Optimized model selection for production deployment")
    print(f"• Comprehensive analysis reduces model selection uncertainty")
    print(f"• Advanced visualizations support better decision-making")

create_improvement_summary()


## 8. Usage Instructions and Next Steps


In [ ]:
print("=" * 80)
print("📖 USAGE INSTRUCTIONS")
print("=" * 80)

print("""
🚀 HOW TO USE THIS ENHANCED ANALYSIS:

1. DATA PREPARATION:
   - Ensure 'news_data.csv' is in the same directory
   - The script automatically selects the 4th most frequent stock
   - Modify TARGET_STOCK variable to analyze different stocks

2. CUSTOMIZATION OPTIONS:
   - Adjust epoch configurations in ModelTrainingPipeline.__init__()
   - Modify feature sets in EnhancedDataPreparation class
   - Change sentiment model weights in MultiSentimentAnalyzer
   - Add new LSTM architectures in MultiLSTMModels class

3. RUNNING THE ANALYSIS:
   - Execute cells sequentially from top to bottom
   - Monitor training progress in the console output
   - Review comprehensive reports and visualizations

4. INTERPRETING RESULTS:
   - Check the final report for best model recommendations
   - Use performance heatmaps to understand model behavior
   - Analyze sentiment dashboard for news impact insights
   - Review epoch analysis for optimal training duration

5. PRODUCTION DEPLOYMENT:
   - Use the best performing model configuration
   - Implement the recommended feature set
   - Set up automated retraining pipeline
   - Monitor model performance over time

⚠️  IMPORTANT NOTES:
- Training 60 configurations may take 30-60 minutes depending on hardware
- GPU acceleration is recommended for faster training
- Ensure sufficient memory for large datasets
- Results may vary based on the selected stock and time period

🔧 CUSTOMIZATION EXAMPLES:
- To test different epochs: modify self.epoch_configs = [25, 50, 75, 100]
- To add new sentiment models: extend MultiSentimentAnalyzer class
- To create new LSTM architectures: add methods to MultiLSTMModels class
- To modify features: update prepare_*_features methods

📊 OUTPUT FILES:
- All results are stored in training_pipeline.results dictionary
- Trained models are saved in training_pipeline.trained_models
- Performance data is available in final_report_df DataFrame
""")

print("\n" + "=" * 80)
print("🎉 ENHANCED STOCK MARKET ANALYSIS COMPLETED!")
print("=" * 80)
print(f"✅ Successfully analyzed {TARGET_STOCK} with comprehensive methodology")
print(f"✅ Tested {len(final_report_df) if 'final_report_df' in locals() else 'N/A'} model configurations")
print(f"✅ Generated advanced visualizations and detailed reports")
print(f"✅ Provided actionable recommendations for production deployment")
print("=" * 80)